In [9]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import mediapipe as mp

In [19]:
cap = cv2.VideoCapture('../PUBG3.mp4')


if cap is None : 
    print('Video read failed')
    sys.exit()
                    
    
while True : 
    ret, frame = cap.read()
    
    if ret == False :
        break
    
    if cv2.waitKey(10) == 27 :
        break
        
    resize_frame = cv2.resize(frame, (1280, 720), interpolation=cv2.INTER_CUBIC)
    cv2.imshow('video', resize_frame)   
    
cap.release()
cv2.destroyAllWindows()

In [13]:
cap = cv2.VideoCapture("../PUBG3.mp4")

# 배경 제거 객체 생성

fgbg = cv2.bgsegm.createBackgroundSubtractorMOG()

 
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 배경 제거 마스크 계산

    fgmask = fgbg.apply(frame)
    cv2.imshow('frame', frame)
   # cv2.imshow('bg', fgmask)
    resize_frame = cv2.resize(frame, (1280, 720), interpolation=cv2.INTER_CUBIC)
    if cv2.waitKey(1) & 0xff == 27:

        break

cap.release()
cv2.destroyAllWindows()

 BackgroundSubtractorMOG2 

In [14]:
cap = cv2.VideoCapture("../PUBG3.mp4")

# 옵션 설명 http://layer0.authentise.com/segment-background-using-computer-vision.html
fgbg = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=500, detectShadows=0)


while(1):
    ret, frame = cap.read()

    width = frame.shape[1]
    height = frame.shape[0]
    frame = cv2.resize(frame, (int(width/3), int(height/3)))

    fgmask = fgbg.apply(frame)

    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(fgmask)


    for index, centroid in enumerate(centroids):
        if stats[index][0] == 0 and stats[index][1] == 0:
            continue
        if np.any(np.isnan(centroid)):
            continue


        x, y, width, height, area = stats[index]
        centerX, centerY = int(centroid[0]), int(centroid[1])

        if area > 10:
            
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 0, 255))


    cv2.imshow('mask',fgmask)
    cv2.imshow('frame',frame)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()



AttributeError: 'NoneType' object has no attribute 'shape'

In [11]:
cap = cv2.VideoCapture("../PUBG3.mp4")

# 옵션 설명 http://layer0.authentise.com/segment-background-using-computer-vision.html
fgbg = cv2.createBackgroundSubtractorMOG2(varThreshold=100)


while(1):
    ret, frame = cap.read()

    fgmask = fgbg.apply(frame)

    nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(fgmask)


    for index, centroid in enumerate(centroids):
        if stats[index][0] == 0 and stats[index][1] == 0:
            continue
        if np.any(np.isnan(centroid)):
            continue


        x, y, width, height, area = stats[index]
        centerX, centerY = int(centroid[0]), int(centroid[1])

        if area > 100:
            cv2.circle(frame, (centerX, centerY), 1, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 0, 255))
            resize_frame = cv2.resize(frame, (1280, 720), interpolation=cv2.INTER_CUBIC)
            cv2.imshow('video', resize_frame)   
 

    cv2.imshow('mask',fgmask)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

In [3]:
cap = cv2.VideoCapture('../PUBG3.mp4')
mp_drawing = mp.solutions.drawing_utils
mp_objectron = mp.solutions.objectron

with mp_objectron.Objectron(static_image_mode=False,
                            max_num_objects=5,
                            min_detection_confidence=0.5,
                            min_tracking_confidence=0.99,
                            model_name='Shoe') as objectron:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("카메라를 찾을 수 없습니다.")
            break
            
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = objectron.process(image)
        
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.detected_objects:
            for detected_object in results.detected_objects:
                mp_drawing.draw_landmarks(
                  image, detected_object.landmarks_2d, mp_objectron.BOX_CONNECTIONS)
                mp_drawing.draw_axis(image, detected_object.rotation,
                                     detected_object.translation)

if cap is None : 
    print('Video read failed')
    sys.exit()
                    
    
while True : 
    ret, frame = cap.read()
    
    if ret == False :
        break
    
    if cv2.waitKey(10) == 27 :
        break
        
    resize_frame = cv2.resize(frame, (1280, 720), interpolation=cv2.INTER_CUBIC)
    # cv2.imshow('video', resize_frame)   
    cv2.imshow('MediaPipe Objectron', cv2.flip(image, 1))
 
cap.release()
cv2.destroyAllWindows()

카메라를 찾을 수 없습니다.


In [13]:
model='../googlenet/opencv_face_detector_uint8.pb'
config='../googlenet/opencv_face_detector.pbtxt.txt'

face_net=cv2.dnn.readNet(model,config)

cap=cv2.VideoCapture('../PUBG3.mp4')
if not cap.isOpened():
    print('video open failed')
    sys.exit()

while True:
    ret,frame=cap.read()
    if not ret:
        print('frame read failed')
        break
    blob = cv2.dnn.blobFromImage(frame, 1, (300, 300), (104, 177, 123),swapRB=False)
    face_net.setInput(blob)
    out=face_net.forward()

    detect=out[0,0,:,:]

    h,w=frame.shape[:2]
    for i in range(detect.shape[0]):
        confidence=detect[i,2]
        if confidence > 0.5:
            x1=int(detect[i,3]*w)
            y1=int(detect[i,4]*h)
            x2=int(detect[i,5]*w)
            y2=int(detect[i,6]*h)
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,125,255),2)

    resize_frame = cv2.resize(frame, (1280, 720), interpolation=cv2.INTER_CUBIC)
    cv2.imshow('image',resize_frame)
    
    if cv2.waitKey(20)==27:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
filename = "full_body8.png"
# 예측할 그림 가져오기, gray convert
image1 = cv2.imread('opencv_data\\'+filename)     
grayImage1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)

image2 = cv2.imread('opencv_data\\'+filename)     
grayImage2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

image3 = cv2.imread('opencv_data\\'+filename)     
grayImage3 = cv2.cvtColor(image3, cv2.COLOR_BGR2GRAY)

# cascade xml 파일 선택
body_cascade = cv2.CascadeClassifier('opencv_data\\haarcascade_fullbody.xml')


# 10 = 검출한 사각형 사이 최소 간격, body에 x,y,w,h가 여러개 저장됨.
body = body_cascade.detectMultiScale(grayImage1, 1.01, 10, minSize=(30, 30))

for (x,y,w,h) in body :         
    cv2.rectangle(image1,(x,y),(x+w,y+h),(0,0,255),3)

plt.subplot(1, 3, 1)
plt.imshow(image1, cmap='gray')


# 2 = 검출한 사각형 사이 최소 간격, minSize = 지정한 크기 이상의 사각형만 가져온다.
body = body_cascade.detectMultiScale(grayImage2,1.01, 2, 0, minSize=(70, 70))

for (x,y,w,h) in body :        
    cv2.rectangle(image2,(x,y),(x+w,y+h),(0,0,255),3)

plt.subplot(1, 3, 2)
plt.imshow(image2, cmap='gray')


# minSize가 없을 때의 결과
body = body_cascade.detectMultiScale(grayImage3, 1.01, 2)
for (x,y,w,h) in body : 
    cv2.rectangle(image3,(x,y),(x+w,y+h),(0,0,255),3)

plt.subplot(1, 3, 3)
plt.imshow(image3, cmap='gray')


plt.show()

In [12]:
cap = cv2.VideoCapture('../PUBG.mp4')

body_cascade = cv2.CascadeClassifier('../googlenet/haarcascade_fullbody.xml')

if cap is None : 
    print('Video read failed')
    sys.exit()
                    
while True : 
    ret, frame = cap.read()
    grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    body = body_cascade.detectMultiScale(grayframe, 1.01, 10, minSize=(30, 30))
    for (x,y,w,h) in body :         
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),3, 4, 0)
    
    #cv2.imshow('body', frame)
    
    if ret == False :
        break
    
    if cv2.waitKey(10) == 27 :
        break
        
    resize_frame = cv2.resize(frame, (960, 640), interpolation=cv2.INTER_CUBIC)
    cv2.imshow('video', resize_frame)   
    
    
cap.release()
cv2.destroyAllWindows()